Ovo je model koji služi za prepoznavanje izgleda bolida tima u Formuli 1 na osnovu date slike. Ukupno postoji 10 timova: Mercedes, McLaren, Williams, Red Bull, Racing Bulls, Kick Sauber, Ferrari, Haas, Alpine i Aston Martin. Dataset ukupno sadrži 983 slike bolida. Kao baza ovog modela koristi se pretrenirani model ResNet50 koji je pogodan za detekciju slika. U prvom koraku koristi se tzv. ekstrakcija osobina (feature extraction), gde se izlazi iz baznog modela koriste kao ulazi za dodatne, sopstvene slojeve modela. Nakon toga otključavamo bazu modela radi tzv. fine-tuninga, tj. dodatnog treniranja celog modela kako bi se bolje prilagodio konkretnom zadatku i poboljšao preciznost.

In [15]:
!pip -q install tensorflow

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Dataset.zip to Dataset.zip


In [ ]:
import os
import zipfile

zip_path = 'Dataset.zip'
extract_path = 'dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [37]:
# Učitavanje slika iz foldera, automatski određujući klase prema podfolderima

import keras
import tensorflow as tf
import numpy as np

dataset = keras.utils.image_dataset_from_directory(
    'dataset/Dataset', # Putanja do dataset-a
    image_size = (224,224), # Dimenzije na koje će sve slike biti skalirane
    batch_size = 32 # Broj slika po batch-u
    )

print(f'Classes: {dataset.class_names}' )

number_of_classes = len(dataset.class_names) # Ukupan broj klasa, odnosno broj izlaza

# Postupak razdvajanja na slike(X) i labele(Y)

X = []
Y = []

for image,label in dataset:
  X.append(image)
  Y.append(label)

# Spajanje tensor-a, kako bi mogao da se vrši trening

X = tf.concat(X, axis = 0)
Y = tf.concat(Y, axis = 0)

Found 983 files belonging to 10 classes.
Classes: ['Alpine', 'Aston Martin', 'Ferrari', 'Haas', 'Kick Sauber', 'McLaren', 'Mercedes', 'Racing Bulls', 'Red Bull', 'Williams']


In [38]:
# Deljenje podataka na trening i test set

from sklearn.model_selection import train_test_split

X_np = X.numpy()
Y_np = Y.numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X_np, Y_np, test_size = 0.2, random_state = 42)

In [39]:
# Kreiranje normalizacionog sloja koji vrši normalizacija na ulazne podatke, prema ResNet50 uputstvima

normalization_layer = keras.layers.Lambda(keras.applications.resnet50.preprocess_input)

# Kreiranje augmentacionog sloja, kako bi model bolje generalizovao i kako bi se povećao broj ulaznih podataka

data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.05),
    keras.layers.RandomZoom(0.05),
    keras.layers.RandomContrast(0.05),
    keras.layers.RandomBrightness(0.05)
])

In [40]:
# Učitavanje RestNet50 modela, ali bez poslednjeg sloja

base_model = keras.applications.ResNet50(weights = "imagenet", input_shape = (224,224,3), include_top = False)
base_model.trainable = False

# Kreiranje modela

model = keras.Sequential([
    data_augmentation,
    normalization_layer,
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(1024, activation="relu"),
    keras.layers.Dense(1024,activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(number_of_classes, activation = "softmax")
])

# Kompajliranje modela

model.compile(optimizer = keras.optimizers.Adam(learning_rate=1e-4), metrics = ["accuracy"], loss = keras.losses.SparseCategoricalCrossentropy())


In [41]:
# Prikaz arhitekture modela

model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_7 (Sequential)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_2 (Lambda)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [42]:
# Definisanje callback funkcije u slučaju da model stagnira sa treniranjem

callbacks = [keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)]

# Treniranje modela

model.fit(X_train, Y_train, batch_size = 64, epochs = 20, validation_split=0.1, callbacks=callbacks)

Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 596ms/step - accuracy: 0.1346 - loss: 2.4626 - val_accuracy: 0.4557 - val_loss: 1.8407
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 309ms/step - accuracy: 0.3346 - loss: 1.9140 - val_accuracy: 0.5823 - val_loss: 1.6136
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 318ms/step - accuracy: 0.4940 - loss: 1.5792 - val_accuracy: 0.6456 - val_loss: 1.3864
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 287ms/step - accuracy: 0.6051 - loss: 1.3300 - val_accuracy: 0.6835 - val_loss: 1.1962
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 315ms/step - accuracy: 0.6228 - loss: 1.1815 - val_accuracy: 0.7342 - val_loss: 1.0300
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 296ms/step - accuracy: 0.6478 - loss: 1.0596 - val_accuracy: 0.7468 - val_loss: 0.9555
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 290ms/step - accuracy: 0.6983 - loss: 0.9820 - val_accuracy: 0.7468 - val_loss: 0.8332
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 289ms/step - accuracy: 0.7539 - loss: 0.8215 - val_accuracy: 0

Fine tuning

In [43]:
# Nakon osnovnog treniranja otključavaju se slojevi baznog modela i primenjuje se fine tuning

#for layer in base_model.layers[100:]:
  #layer.trainable = True

base_model.trainable = True

# Kompajliranje tog novog modela

model.compile(optimizer = keras.optimizers.Adam(1e-5), loss = keras.losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])

# Treniranje modela

model.fit(X_train,Y_train, batch_size = 64, epochs = 15, callbacks=callbacks, validation_split=0.1)


Epoch 1/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.7532 - loss: 0.7465 - val_accuracy: 0.7722 - val_loss: 0.5419
Epoch 2/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 977ms/step - accuracy: 0.9046 - loss: 0.3709 - val_accuracy: 0.7342 - val_loss: 0.6114
Epoch 3/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 994ms/step - accuracy: 0.9334 - loss: 0.2925 - val_accuracy: 0.7722 - val_loss: 0.5279
Epoch 4/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 976ms/step - accuracy: 0.9242 - loss: 0.2739 - val_accuracy: 0.8228 - val_loss: 0.4551
Epoch 5/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 996ms/step - accuracy: 0.9572 - loss: 0.2277 - val_accuracy: 0.8481 - val_loss: 0.4253
Epoch 6/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 982ms/step - accuracy: 0.9632 - loss: 0.1854 - val_accuracy: 0.8608 - val_loss: 0.3584
Epoch 7/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 989ms/step - accuracy: 0.9711 - loss: 0.1685 - val_accuracy: 0.9114 - val_loss: 0.2782
Epoch 8/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 1000ms/step - accuracy: 0.9762 - loss: 0.1725 - val_accura

In [44]:
# Određivanje predikcija na test set-u

y_pred_probs = model.predict(X_test)

# Dobijene verovatnoće se prebacuju u stvarne klase

y_pred = np.argmax(y_pred_probs, axis=1)

print(y_pred)

7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 394ms/step
[8 9 2 2 0 0 6 4 8 6 2 8 2 3 6 9 4 9 5 5 2 7 4 2 3 8 0 8 2 5 9 5 2 2 4 9 6
 1 1 0 6 5 0 9 1 1 1 5 7 8 7 4 3 8 2 1 1 8 5 0 9 8 9 2 5 8 6 0 3 0 8 8 8 7
 9 0 1 8 2 4 0 8 8 7 6 9 9 2 6 0 9 0 1 0 8 3 1 3 4 3 3 2 3 1 5 8 8 3 5 2 6
 6 1 4 6 8 4 7 8 0 6 9 0 8 8 5 1 0 8 0 3 0 2 5 9 7 2 6 9 7 9 6 0 4 4 2 7 3
 3 2 8 0 8 5 7 6 7 2 4 1 2 6 0 3 9 1 6 4 0 1 9 8 2 3 3 6 3 7 7 1 9 7 5 5 8
 4 8 5 2 4 6 3 1 1 4 0 5]


In [45]:
# Računanje tačnosti

accurate = np.sum(y_pred == Y_test)
total = len(Y_test)

print(f'Accuracy: {accurate/total}')

Accuracy: 0.9390862944162437


In [46]:
# Prikazivanje statistike po klasama

from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred, target_names=dataset.class_names))

              precision    recall  f1-score   support

      Alpine       1.00      0.96      0.98        24
Aston Martin       0.89      0.94      0.92        18
     Ferrari       1.00      0.96      0.98        24
        Haas       0.89      1.00      0.94        16
 Kick Sauber       1.00      0.94      0.97        17
     McLaren       1.00      0.89      0.94        19
    Mercedes       0.89      0.89      0.89        19
Racing Bulls       1.00      0.82      0.90        17
    Red Bull       0.86      0.96      0.91        26
    Williams       0.89      1.00      0.94        17

    accuracy                           0.94       197
   macro avg       0.94      0.94      0.94       197
weighted avg       0.94      0.94      0.94       197

